# Получение сырых данных из Яндекс Метрики

### Подготовка к работе

**Описание таблиц**  
Документация API Metrika: https://tech.yandex.ru/metrika/doc/api2/concept/about-docpage/  
Просмотры: https://tech.yandex.ru/metrika/doc/api2/logs/fields/hits-docpage/  
Визиты:    https://tech.yandex.ru/metrika/doc/api2/logs/fields/visits-docpage/

# Загрузка библиотек

In [ ]:
import requests
import json
import pandas as pd
from io import StringIO

# Функции

## создание нового запроса

In [ ]:
def make_post_request(source, token, counterId, dt1, dt2, col):
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests'
    payload = {
    'date1': dt1,
    'date2': dt2,
    'fields': col,
    'source' : source
    }
    header = {'Authorization': f'OAuth {token}'}
    
    post = requests.post(url,data=payload, headers=header)
    # печатаем ответ, обращаем внимание на параметры:
    # request_id - id нашего запроса (нужен для скачивания информации)
    # status - показывает создан ли наш лог запросов ("status":"created")
    logs = json.loads(post.text)
    print('Cоздан запрос id: {}, со статусом: {}'.format(logs['log_request']['request_id'], logs['log_request']['status']))
    return post.text

## список ранее созданных запросов

In [ ]:
def get_all_id_requests(counterId, token):
    # получаем все id запросов на сервере
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests'
    
    header = {'Authorization': f'OAuth {token}'}
    all_requests = requests.get(url, headers=header)

    
    logs = json.loads(all_requests.text)
    res = [log['request_id'] for log in logs['requests']]
    return res#, logs

## удаление ранее созданных запросов

In [ ]:
def clean_all_request(counterId, token):
    header = {'Authorization': f'OAuth {token}'}
    # удаляем все запросы на сервере
    id_list = get_all_id_requests(counterId, token)#[0]
    if not id_list:
        print('На сервере нет логов')
        return None
    for requestId in id_list:
        url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}/clean'
        post = requests.post(url, headers=header)
        print(f'Запрос id {requestId} удален...')
    return None

## статус запроса

In [ ]:
def check_request_status(counterId, token, requestId):
    # проверяем доступен ли наш запрос для скачивания логов ("status":"processed")
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}'
    header = {'Authorization': f'OAuth {token}'}
    
    all_requests = requests.get(url, headers=header)
    
    logs = json.loads(all_requests.text)
    
    status = logs['log_request']['status']
    return requestId, status

## создание df

In [ ]:
def request_to_dataframe(counterId, token, requestId, partNumber=0):
    url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{requestId}/part/{partNumber}/download'
    header = {'Authorization': f'OAuth {token}'}
    r = requests.get(url, headers=header)
    df = pd.read_csv(StringIO(r.text), header=0, sep='\t')
    return df

In [ ]:
# функция составления строки с перечнем столбцов для нового запроса
def columns(df, index_list):
    if len(index_list) >0:
        col = df.loc[0][0]
        for i in (index_list[1:]):
            col = col + ',' + df.loc[i][0]
        return col
    else:
        col = df.loc[0][0]
        for i in range(1,df.shape[0]):
            col = col + ',' + df.loc[i][0]
        return col

# Основной блок

## Устанавливаем значения

In [ ]:
# Устанавливаем значения для запроса:
# token     - токен для получения информации
# counterId - номер счетчика Яндекс Метрики (если несколько, указываются через запятую)

token = ''
counterId = 

### выбор полей(столбцов)

In [ ]:
# посмотреть список полей с описанием
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    display(x)
    pd.reset_option('display.max_rows')

#### visits

In [ ]:
from io import BytesIO

spreadsheet_id = '1aW4sN9vDBnL76X9WxBhx9nt0xvq8FKlgk8B1ArGxne4'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
vis = pd.read_csv(BytesIO(r.content))

vis.columns = vis.columns.str.replace(' ','')
vis['Поле'] = vis['Поле'].str.replace(' ','')

In [ ]:
# список индексов необходимых полей
num_visits = [] # [0,1,2,3,6,7,8,9,10,11,13,14,17,18,27,28,30,61,62,65,66]
str_visits = columns(vis, num_visits)
str_visits

#### hits

In [ ]:
from io import BytesIO

spreadsheet_id = '1UW4VJI1BXy8q_T3R1dfbz3vKO9apyoJmlWycelHvUVU'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
hits = pd.read_csv(BytesIO(r.content))

hits.columns = hits.columns.str.replace(' ','')
hits['Поле'] = hits['Поле'].str.replace(' ','')

In [ ]:
# список индексов необходимых полей
num_hits = [] # [0,1,2,3,6,7,8,9,10,11,13,14,17,18,27,28,30,61,62,65,66]
str_hits = columns(hits, num_hits)

## Получаем Id запросов, существующих на сервере Logs API 

In [ ]:
# получаем все идентификаторы запросов на сервере
get_all_id_requests(counterId, token)

## Очищаем список запросов на сервере Logs API

In [ ]:
# удаляем запросы на сервере Logs API (если сущетсвуют)
# опционально
clean = clean_all_request(counterId, token)
clean

## Создаем новый запрос 'visits' или 'hits'

In [ ]:
# создаем новый запрос логов/// не работает Major-версия браузера	Minor-версия браузера

# dt1, dt2  - начальная и конечная (не может быть текущим днем) даты формата 'YYYY-MM-DD'
# source    - источинк данных данных для запроса ('hits' или 'visits')
dt1, dt2 = '2022-06-01', '2022-07-25'
source = 'visits'

col = str_visits if source == 'visits' else str_hits
query1 = make_post_request(source=source, token=token, counterId=counterId, dt1=dt1, dt2=dt2, col=col)

## Проверяем статус нужного запроса по requestId

In [ ]:
# проверяем готовность запроса к скачиванию
check1 = check_request_status(counterId, token, requestId='28231771')
# статус дложен быть 'processed'!
check1

## Записываем файл в формате DataFrame пакета pandas

In [ ]:
df = request_to_dataframe(counterId=counterId, token=token, requestId='28231771')
df.head()

## Записываем в файл формата csv, если требуется открыть в Excel  
Записывается в ту же папку, где находится данный блокнот

In [ ]:
# записываем DataFrame в csv
df.to_csv(f'{source}_{df['date'].min()}_{df['date'].max()}.csv', sep='\t')#, encoding='cp1251')